In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Placeholder for dataset loading (using synthetic data for simplicity)
def generate_synthetic_data(num_samples, grid_size=32):
    return np.random.randint(0, 2, size=(num_samples, grid_size, grid_size, grid_size))

# Example: Generate 1000 samples of 3D voxel grids
X_train = generate_synthetic_data(1000)
X_train = X_train.astype('float32')  # Normalize the data

# Visualize a sample voxel grid
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.voxels(X_train[0], edgecolors='k')
plt.show()

In [3]:
# Encoder network
def build_encoder(latent_dim=64, input_shape=(32, 32, 32, 1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv3D(32, 3, activation='relu', strides=2, padding='same')(inputs)
    x = layers.Conv3D(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    
    # Latent space (mean and variance)
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    
    return models.Model(inputs, [z_mean, z_log_var])

# Decoder network
def build_decoder(latent_dim=64, output_shape=(32, 32, 32, 1)):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(128, activation='relu')(latent_inputs)
    x = layers.Dense(8 * 8 * 8 * 64, activation='relu')(x)
    x = layers.Reshape((8, 8, 8, 64))(x)
    x = layers.Conv3DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv3DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv3DTranspose(1, 3, activation='sigmoid', padding='same')(x)
    
    return models.Model(latent_inputs, x)

# VAE Model combining Encoder and Decoder
latent_dim = 64
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)

# Sampling function for reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define the VAE model
inputs = layers.Input(shape=(32, 32, 32, 1))
z_mean, z_log_var = encoder(inputs)
z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
decoded = decoder(z)

vae = models.Model(inputs, decoded)

# Loss function: Reconstruction + KL Divergence
def vae_loss(y_true, y_pred):
    reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true, y_pred), axis=(1, 2, 3)))
    kl_loss = - 0.5 * tf.reduce_mean(tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
    return reconstruction_loss + kl_loss

# Compile the model
vae.compile(optimizer='adam', loss=vae_loss)

In [4]:
# Train the VAE
vae.fit(X_train, X_train, epochs=50, batch_size=32)

In [5]:
# Generate and visualize new 3D objects
def generate_3d_objects(decoder, latent_dim, num_samples=5):
    z_samples = np.random.normal(size=(num_samples, latent_dim))
    generated_3d_objects = decoder.predict(z_samples)
    return generated_3d_objects

# Generate and visualize new 3D objects
generated_objects = generate_3d_objects(decoder, latent_dim)

# Visualize the first generated object
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.voxels(generated_objects[0].reshape(32, 32, 32), edgecolors='k')
plt.show()